<a href="https://colab.research.google.com/github/ARJUN108-verma/Streamlit-/blob/main/app2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#app2

In [ ]:
!pip install streamlit

In [ ]:
from openai import OpenAI
import streamlit as st

In [ ]:
# Setting up the Streamlit page configuration
st.set_page_config(page_title="Streamlit Chat", page_icon="💬")
st.title("Chatbot")


In [ ]:
# Personal Information Section
st.subheader('Personal information', divider='rainbow')

In [ ]:
# Input fields for collecting user's personal information
name = st.text_input(label = "Name", max_chars = None, placeholder = "Enter your name")

In [ ]:
experience = st.text_area(label = "Expirience", value = "", height = None, max_chars = None, placeholder = "Describe your experience")


In [ ]:
skills = st.text_area(label = "Skills", value = "", height = None, max_chars = None, placeholder = "List your skills")

In [ ]:
# Test labels for personal information
st.write(f"**Your Name**: {name}")
st.write(f"**Your Experience**: {experience}")
st.write(f"**Your Skills**: {skills}")


In [ ]:
# Company and Position Section
st.subheader('Company and Position', divider = 'rainbow')


In [ ]:
#Field for selecting the job level, position and company
col1, col2 = st.columns(2)
with col1:
    level = st.radio(
    "Choose level",
    key="visibility",
    options=["Junior", "Mid-level", "Senior"],
    )

with col2:
    position = st.selectbox(
    "Choose a position",
    ("Data Scientist", "Data engineer", "ML Engineer", "BI Analyst", "Financial Analyst"))

company = st.selectbox(
    "Choose a Company",
    ("Amazon", "Meta", "Udemy", "365 Company", "Nestle", "LinkedIn", "Spotify")
)

In [ ]:
# Test labels for company and position information
st.write(f"**Your information**: {level} {position} at {company}")

In [ ]:
# Initializing the OpenAI client using the API key from Streamlit's secrets
client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"])

In [ ]:
# Setting up the OpenAI model in session state if it is not already defined
if "openai_model" not in st.session_state:
    st.session_state["openai_model"] = "gpt-4o"


In [ ]:
# Initializing the 'messages' list and adding a system message
if "messages" not in st.session_state:
    st.session_state.messages = [{"role":"system", "content": f"You are an HR executive that interviews an interviewee called {name} with expirience {experience} and skills {skills}. You should interview him for the position {level} {position} at the company {company}"}]

In [ ]:
# Looping through the 'messages' list to display each message except system messages
for message in st.session_state.messages:
    if message["role"] != "system":
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

In [ ]:
# Input field for the user to send a new message
if prompt := st.chat_input("Your answer."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # Assistant's response
    with st.chat_message("assistant"):
        stream = client.chat.completions.create(
            model=st.session_state["openai_model"],
            messages=[
                {"role": m["role"], "content": m["content"]}
                for m in st.session_state.messages
            ],
            stream=True,
        )
        # Display the assistant's response as it streams
        response = st.write_stream(stream)
    # Append the assistant's full response to the 'messages' list
    st.session_state.messages.append({"role": "assistant", "content": response})